# Webscrapping using requests and BeautifulSoup

Country Information Web Scraper
This is a Python script that extracts country information from a local web page and stores it in a DataFrame. The script uses urllib, BeautifulSoup and pandas libraries.

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime

In [7]:
def scrap_Element(id_info):
    country_info = country_soup.find("tr",id=id_info).find("td",class_="w2p_fw")
    country_info =str(country_info).split(">")[1]
    country_info = country_info.split("<")[0]
    return country_info

In [8]:
""" pseudo-code solution
access first page
check if it has content
while it has content on the page
  make the list of links on the page
  for each link on the page
    access the page
    get the country information
    store in a row vector
    store the row vector in a countries matrix
  access next page
  check if the page has content
"""

index = 0

#link é uma string que contém a URL para acessar a página de índice
link = "http://127.0.0.1:8000/places/default/index/"+str(index)

#site é uma variável que faz uma solicitação para a URL usando urllib e armazena o conteúdo da página
site = urllib.request.urlopen(link)

#soup rmazena uma representação analisada da página HTML usando o BeautifulSoup.
soup = BeautifulSoup(site,"html.parser")

#Countries é uma lista que será usada para armazenar informações sobre os países.
countries = []

#country_link é uma lista de elementos que contem o link da página de cada país.
country_link = soup.find_all("a",href=re.compile("/places/default/view/"))

while len(country_link) > 0:
    #Para cada  o link faz a requisição GET para a página
    #A partir deste ponto, o código faz extrações específicas de informações, como nome do país, capital, 
    #moeda e população de cada país, para adicionar essas informações à lista
    
    #Obtendo a lista de links
    for link in country_link:
        aux = str(link)
        aux = aux.split("><")[0]
        aux = aux.split("/")[-1]
        aux = aux[:-1]
        country_page = requests.get("http://127.0.0.1:8000"+"/places/default/view/"+aux)

        country = []
        country_soup = BeautifulSoup(country_page.text,"html.parser")

        #Nome do país
        id_info = "places_country__row"
        country.append(scrap_Element(id_info))

        #Capital
        id_info = "places_capital__row"
        country.append(scrap_Element(id_info))  
        
        #Moeda
        id_info = "places_currency_name__row"
        country.append(scrap_Element(id_info))
        
        #Populacao
        id_info = "places_population__row"
        country.append(scrap_Element(id_info))

        country.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        countries.append(country)

    index +=1
    link = "http://127.0.0.1:8000/places/default/index/"+str(index)
    site = urllib.request.urlopen(link)
    soup = BeautifulSoup(site,"html.parser")
    country_link = soup.find_all("a",href=re.compile("/places/default/view/"))

In [9]:
#Cria um DataFrame do Pandas chamado df_countries a partir da lista countries que foi preenchida com informações sobre os países. 
#Ele também define os nomes das colunas do DataFrame como "Nome", "Capital", "Moeda", "População" e "Date Time". 
df_countries =  pd.DataFrame(countries,columns=["Nome","Capital","Moeda","População","Date Time"])
display(df_countries)

,Nome,Capital,Moeda,População,Date Time
0,Afghanistan,Kabul,Afghani,29121286,2023-09-06 16:50:00
1,Aland Islands,Mariehamn,Euro,26711,2023-09-06 16:50:01
2,Albania,Tirana,Lek,2986952,2023-09-06 16:50:01
3,Algeria,Algiers,Dinar,34586184,2023-09-06 16:50:01
4,American Samoa,Pago Pago,Dollar,57881,2023-09-06 16:50:02
...,...,...,...,...,...
247,Wallis and Futuna,Mata Utu,Franc,16025,2023-09-06 16:51:20
248,Western Sahara,El-Aaiun,Dirham,273008,2023-09-06 16:51:21
249,Yemen,Sanaa,Rial,23495361,2023-09-06 16:51:21
250,Zambia,Lusaka,Kwacha,13460305,2023-09-06 16:51:22


In [10]:
#Conversão do DF para um CSV
df_countries.to_csv("countries.csv",index=False)